In [31]:
import pandas as pd
import numpy as np

In [32]:
train_values = pd.read_csv('../data/raw/training_values.csv')
train_labels = pd.read_csv('../data/raw/training_labels.csv')
train_df = train_values.merge(train_labels, on='id')

test_values = pd.read_csv('../data/raw/test_values.csv')

Many features have illogical values (e.g. '0' as the name of the funder).  I am replacing these illogical values with nans, as the values were likely entered as a placeholder for 'unknown'.

In [33]:
clean_train = train_df
clean_test = test_values

for df in [clean_train, clean_test]:
    df['funder'].replace('0', np.nan, inplace=True)
    df['installer'].replace('0', np.nan, inplace=True)
    df['longitude'].replace(0, np.nan, inplace=True)
    df['latitude'].replace(-0.00000002, np.nan, inplace=True)
    df['population'].replace(0, np.nan, inplace=True)
    df['construction_year'].replace(0, np.nan, inplace=True)

Some of the test features contain different conventions for capitalization (e.g. District Council vs. District council).  Thus, I am making all text values lowercase to ensure entries are consistent.

In [34]:
for df in [clean_train, clean_test]:
    for col in df:
        if df[col].dtype == 'O' and col not in ['permit', 'public_meeting']:
            df[col] = df[col].str.lower()

Making the date recorded datetime, so that alogrithms can split on before/after a certain date.

In [35]:
clean_train['date_recorded'] = pd.to_datetime(clean_train['date_recorded'])
clean_test['date_recorded'] = pd.to_datetime(clean_test['date_recorded'])

Making new column that aggregates all other text columns.  This will be passed into a naive bayes classifier to be used as a feature later on.  The goal of using this technique is to capture some of the signal held by these columns without making dummies for all of them.  Not all columns are included, as some seem to add no new information.  The recorded_by column has the same value for every entry, thus is adding no information.

In [36]:
clean_train['text_cols'] = clean_train[['funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 'scheme_management', 'scheme_name', 'extraction_type', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'waterpoint_type']].values.tolist()
clean_train.drop(['funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 'scheme_management', 'scheme_name', 'extraction_type', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'waterpoint_type', 'recorded_by', 'extraction_type_group', 'extraction_type_class', 'quantity_group', 'source_class', 'waterpoint_type_group'], axis=1, inplace=True)

clean_test['text_cols'] = clean_test[['funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 'scheme_management', 'scheme_name', 'extraction_type', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'waterpoint_type']].values.tolist()
clean_test.drop(['funder', 'installer', 'wpt_name', 'basin', 'subvillage', 'region', 'lga', 'ward', 'scheme_management', 'scheme_name', 'extraction_type', 'management', 'management_group', 'payment', 'payment_type', 'water_quality', 'quality_group', 'quantity', 'source', 'source_type', 'waterpoint_type', 'recorded_by', 'extraction_type_group', 'extraction_type_class', 'quantity_group', 'source_class', 'waterpoint_type_group'], axis=1, inplace=True)

In [ ]:
clean_train['text_cols'] = clean_train['text_cols'].map(lambda x: ' '.join(x))
clean_test['text_cols'] = clean_test